In [ ]:
import helpers
from sklearn import preprocessing
import numpy as np
X_test, y_test = helpers.get_data(subset="test")

In [ ]:
le = preprocessing.OrdinalEncoder()
le.fit(X_test['native-country'].astype(str).values.reshape(-1,1))
le.categories_

In [ ]:
import helpers
from sklearn import preprocessing
import numpy as np
X_test, y_test = helpers.get_data(subset="test")
ohe = preprocessing.OneHotEncoder()
categories = np.array(list(set(X_test['workclass'].astype(str).values))).reshape(-1,1)
ohe.fit(categories)

# OneHotEncoder(categorical_features=None, categories=None,
#     dtype=<class 'numpy.float64'>, handle_unknown='error',
#     n_values=None, sparse=True)

categories

# array([['Self-emp-inc'],
#     ['Local-gov'],
#     ['Private'],
#     ['State-gov'],
#     ['Never-worked'],
#     ['Without-pay'],
#     ['Federal-gov'],
#     ['Self-emp-not-inc'],
#     ['nan']], dtype='<U16')
ohe.transform(categories).todense()

# matrix([[0., 0., 0., 0., 1., 0., 0., 0., 0.],
#     [0., 1., 0., 0., 0., 0., 0., 0., 0.],
#     [0., 0., 0., 1., 0., 0., 0., 0., 0.],
#     [0., 0., 0., 0., 0., 0., 1., 0., 0.],
#     [0., 0., 1., 0., 0., 0., 0., 0., 0.],
#     [0., 0., 0., 0., 0., 0., 0., 1., 0.],
#     [1., 0., 0., 0., 0., 0., 0., 0., 0.],
#     [0., 0., 0., 0., 0., 1., 0., 0., 0.],
#     [0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [ ]:
ohe_encoder_pp = helpers.to_pipeline(("ohe", preprocessing.OneHotEncoder(handle_unknown='ignore', categories=categories.get_categories())))

In [ ]:
ohe_pipeline = pipeline.get_pipeline(ohe_encoder_pp, input_dim=112)

In [ ]:
helpers.execute_on_pipeline(ohe_pipeline, X_train, y_train, X_test, y_test)

In [ ]:
from keras.models import Model
from keras.layers.core import Dense, Reshape, Lambda
from keras.layers import Input, Embedding, merge
from keras import backend as K

# Number of product IDs available
N_products = 1000000
N_stores = 1000
N_shoppers = 10000

# Integer IDs representing 1-hot encodings
prior_in = Input(shape=(1,))
store_in = Input(shape=(1,))
shopper_in = Input(shape=(1,))

# Dense N-hot encoding for candidate products
candidates_in = Input(shape=(N_products,))

# Embeddings
prior = Embedding(N_products, 10)(prior_in)
store = Embedding(N_stores, 10)(store_in)
shopper = Embedding(N_shoppers, 10)(shopper_in)

# Reshape and merge all embeddings together
reshape = Reshape(target_shape=(10,))
combined = merge([reshape(prior), reshape(store), reshape(shopper)],
                 mode='concat')

# Hidden layers
hidden_1 = Dense(1024, activation='relu')(combined)
hidden_2 = Dense(512, activation='relu')(hidden_1)
hidden_3 = Dense(256, activation='relu')(hidden_2)
hidden_4 = Dense(10, activation='linear')(hidden_3)

# Final 'fan-out' into the space of future products
final = Dense(N_products, activation='linear')(hidden_4)

# Ensure we do not overflow when we exponentiate
final = Lambda(lambda x: x - K.max(x))(final)

# Masked soft-max using Lambda and merge-multiplication
exponentiate = Lambda(lambda x: K.exp(x))(final)
masked = merge([exponentiate, candidates_in], mode='mul')
predicted = Lambda(lambda x: x / K.sum(x))(masked)

# Compile with categorical crossentropy and adam
mdl = Model(input=[prior_in, store_in, shopper_in, candidates_in],
            output=predicted)
mdl.compile(loss='categorical_crossentropy', 
            optimizer='adam',
            metrics=['accuracy'])

In [7]:
# example of ordinal encoding for a neural network
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# load the dataset
def load_dataset(filename):
	# load the dataset as a pandas DataFrame
	data = read_csv(filename, header=None)
	# retrieve numpy array
	dataset = data.values
	# split into input (X) and output (y) variables
	X = dataset[:, :-1]
	y = dataset[:,-1]
	# format all fields as string
	X = X.astype(str)
	# reshape target to be a 2d array
	y = y.reshape((len(y), 1))
	return X, y

# prepare input data
def prepare_inputs(X_train, X_test):
	oe = OrdinalEncoder()
	oe.fit(X_train)
	X_train_enc = oe.transform(X_train)
	X_test_enc = oe.transform(X_test)
	return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

# load the dataset
X, y = load_dataset('../input/breast-cancer.csv')
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)
# define the  model
model = Sequential()
model.add(Dense(10, input_dim=X_train_enc.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train_enc, y_train_enc, epochs=100, batch_size=16, verbose=2)
# evaluate the keras model
_, accuracy = model.evaluate(X_test_enc, y_test_enc, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

C:\Users\furyx\miniconda3\envs\tf26\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\furyx\miniconda3\envs\tf26\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/100
12/12 - 0s - loss: 0.9597 - accuracy: 0.7068
Epoch 2/100
12/12 - 0s - loss: 0.9112 - accuracy: 0.6963
Epoch 3/100
12/12 - 0s - loss: 0.8798 - accuracy: 0.7068
Epoch 4/100
12/12 - 0s - loss: 0.8518 - accuracy: 0.6963
Epoch 5/100
12/12 - 0s - loss: 0.8313 - accuracy: 0.7016
Epoch 6/100
12/12 - 0s - loss: 0.8140 - accuracy: 0.7016
Epoch 7/100
12/12 - 0s - loss: 0.7977 - accuracy: 0.7016
Epoch 8/100
12/12 - 0s - loss: 0.7836 - accuracy: 0.7016
Epoch 9/100
12/12 - 0s - loss: 0.7684 - accuracy: 0.7068
Epoch 10/100
12/12 - 0s - loss: 0.7567 - accuracy: 0.7016
Epoch 11/100
12/12 - 0s - loss: 0.7449 - accuracy: 0.7016
Epoch 12/100
12/12 - 0s - loss: 0.7331 - accuracy: 0.7068
Epoch 13/100
12/12 - 0s - loss: 0.7230 - accuracy: 0.7016
Epoch 14/100
12/12 - 0s - loss: 0.7134 - accuracy: 0.6963
Epoch 15/100
12/12 - 0s - loss: 0.7043 - accuracy: 0.7016
Epoch 16/100
12/12 - 0s - loss: 0.6952 - accuracy: 0.7068
Epoch 17/100
12/12 - 0s - loss: 0.6866 - accuracy: 0.7016
Epoch 18/100
12/12 - 0s

How to One Hot Encode Categorical Data

A one hot encoding is appropriate for categorical data where no relationship exists between categories.

It involves representing each categorical variable with a binary vector that has one element for each unique label and marking the class label with a 1 and all other elements 0.

For example, if our variable was “color” and the labels were “red,” “green,” and “blue,” we would encode each of these labels as a three-element binary vector as follows:

    Red: [1, 0, 0]
    Green: [0, 1, 0]
    Blue: [0, 0, 1]

Then each label in the dataset would be replaced with a vector (one column becomes three). This is done for all categorical variables so that our nine input variables or columns become 43 in the case of the breast cancer dataset.

The scikit-learn library provides the OneHotEncoder to automatically one hot encode one or more variables.

The prepare_inputs() function below provides a drop-in replacement function for the example in the previous section. Instead of using an OrdinalEncoder, it uses a OneHotEncoder.

In [6]:
# example of one hot encoding for a neural network
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# load the dataset
def load_dataset(filename):
	# load the dataset as a pandas DataFrame
	data = read_csv(filename, header=None)
	# retrieve numpy array
	dataset = data.values
	# split into input (X) and output (y) variables
	X = dataset[:, :-1]
	y = dataset[:,-1]
	# format all fields as string
	X = X.astype(str)
	# reshape target to be a 2d array
	y = y.reshape((len(y), 1))
	return X, y

# prepare input data
def prepare_inputs(X_train, X_test):
	ohe = OneHotEncoder()
	ohe.fit(X_train)
	X_train_enc = ohe.transform(X_train)
	X_test_enc = ohe.transform(X_test)
	return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

# load the dataset
X, y = load_dataset('../input/breast-cancer.csv')
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)
# define the  model
model = Sequential()
model.add(Dense(10, input_dim=X_train_enc.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train_enc, y_train_enc, epochs=100, batch_size=16, verbose=2)
# evaluate the keras model
_, accuracy = model.evaluate(X_test_enc, y_test_enc, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

C:\Users\furyx\miniconda3\envs\tf26\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\furyx\miniconda3\envs\tf26\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\furyx\miniconda3\envs\tf26\lib\site-packages\tensorflow\python\framework\indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_2/dense_7/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_2/dense_7/embedding_lookup_sparse/Reshape:0", shape=(None, 10), dtype=float32), dense_shape=Tensor("gradient_tape/seque

Epoch 1/100
12/12 - 0s - loss: 0.7343 - accuracy: 0.4607
Epoch 2/100
12/12 - 0s - loss: 0.7010 - accuracy: 0.5445
Epoch 3/100
12/12 - 0s - loss: 0.6718 - accuracy: 0.6073
Epoch 4/100
12/12 - 0s - loss: 0.6472 - accuracy: 0.6649
Epoch 5/100
12/12 - 0s - loss: 0.6268 - accuracy: 0.6963
Epoch 6/100
12/12 - 0s - loss: 0.6105 - accuracy: 0.7120
Epoch 7/100
12/12 - 0s - loss: 0.5960 - accuracy: 0.7277
Epoch 8/100
12/12 - 0s - loss: 0.5852 - accuracy: 0.7330
Epoch 9/100
12/12 - 0s - loss: 0.5749 - accuracy: 0.7382
Epoch 10/100
12/12 - 0s - loss: 0.5673 - accuracy: 0.7435
Epoch 11/100
12/12 - 0s - loss: 0.5596 - accuracy: 0.7435
Epoch 12/100
12/12 - 0s - loss: 0.5533 - accuracy: 0.7435
Epoch 13/100
12/12 - 0s - loss: 0.5472 - accuracy: 0.7435
Epoch 14/100
12/12 - 0s - loss: 0.5417 - accuracy: 0.7539
Epoch 15/100
12/12 - 0s - loss: 0.5370 - accuracy: 0.7539
Epoch 16/100
12/12 - 0s - loss: 0.5324 - accuracy: 0.7539
Epoch 17/100
12/12 - 0s - loss: 0.5278 - accuracy: 0.7592
Epoch 18/100
12/12 - 0s

In [9]:
# example of learned embedding encoding for a neural network
from numpy import unique
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import concatenate
from tensorflow.keras.utils import plot_model

# load the dataset
def load_dataset(filename):
	# load the dataset as a pandas DataFrame
	data = read_csv(filename, header=None)
	# retrieve numpy array
	dataset = data.values
	# split into input (X) and output (y) variables
	X = dataset[:, :-1]
	y = dataset[:,-1]
	# format all fields as string
	X = X.astype(str)
	# reshape target to be a 2d array
	y = y.reshape((len(y), 1))
	return X, y

# prepare input data
def prepare_inputs(X_train, X_test):
	X_train_enc, X_test_enc = list(), list()
	# label encode each column
	for i in range(X_train.shape[1]):
		le = LabelEncoder()
		le.fit(X_train[:, i])
		# encode
		train_enc = le.transform(X_train[:, i])
		test_enc = le.transform(X_test[:, i])
		# store
		X_train_enc.append(train_enc)
		X_test_enc.append(test_enc)
	return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

# load the dataset
X, y = load_dataset('../input/breast-cancer.csv')
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)
# make output 3d
y_train_enc = y_train_enc.reshape((len(y_train_enc), 1, 1))
y_test_enc = y_test_enc.reshape((len(y_test_enc), 1, 1))
# prepare each input head
in_layers = list()
em_layers = list()
for i in range(len(X_train_enc)):
	# calculate the number of unique inputs
	n_labels = len(unique(X_train_enc[i]))
	# define input layer
	in_layer = Input(shape=(1,))
	# define embedding layer
	em_layer = Embedding(n_labels, 10)(in_layer)
	# store layers
	in_layers.append(in_layer)
	em_layers.append(em_layer)
# concat all embeddings
merge = concatenate(em_layers)
dense = Dense(10, activation='relu', kernel_initializer='he_normal')(merge)
output = Dense(1, activation='sigmoid')(dense)
model = Model(inputs=in_layers, outputs=output)
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# plot graph
plot_model(model, show_shapes=True, to_file='embeddings.png')
# fit the keras model on the dataset
model.fit(X_train_enc, y_train_enc, epochs=20, batch_size=16, verbose=2)
# evaluate the keras model
_, accuracy = model.evaluate(X_test_enc, y_test_enc, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/20
12/12 - 1s - loss: 0.6886 - accuracy: 0.6126
Epoch 2/20
12/12 - 0s - loss: 0.6656 - accuracy: 0.7277
Epoch 3/20
12/12 - 0s - loss: 0.6384 - accuracy: 0.7277
Epoch 4/20
12/12 - 0s - loss: 0.6093 - accuracy: 0.7277
Epoch 5/20
12/12 - 0s - loss: 0.5866 - accuracy: 0.7277
Epoch 6/20
12/12 - 0s - loss: 0.5662 - accuracy: 0.7277
Epoch 7/20
12/12 - 0s - loss: 0.5543 - accuracy: 0.7277
Epoch 8/20
12/12 - 0s - loss: 0.5452 - accuracy: 0.7277
Epoch 9/20
12/12 - 0s - loss: 0.5375 - accuracy: 0.7277
Epoch 10/20
12/12 - 0s - loss: 0.5299 - accuracy: 0.7277
Epoch 11/20
12/12 - 0s - loss: 0.5227 - accuracy: 0.7330
Epoch 12/20
12/12 - 0s - loss: 0.5167 - accuracy: 0.7487
Epoch 13/20
12/12 - 0s - loss: 0.5100 - accuracy: 0.7487
Epoch 14/20
12/12 - 0s - loss: 0.5035 - accuracy: 0.7592
Epoch 15/20
12/12 - 0s - loss: 0.4976 - accuracy: 0.7644
Epoch 16/20
12/12 - 0s - loss: 0.4919 - accuracy: 0.7801
Epoch 17/20
12/12 - 0s - loss: 0.4869 - accuracy: 0.7853
Epoch 18/20
12/12 - 0s - loss: 0.4811 - 

In [1]:
import keras_tuner as kt
from sklearn import ensemble
from sklearn import datasets
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn import pipeline

def build_model(hp):
  model_type = hp.Choice('model_type', ['random_forest', 'ridge'])
  if model_type == 'random_forest':
    model = ensemble.RandomForestClassifier(
        n_estimators=hp.Int('n_estimators', 10, 50, step=10),
        max_depth=hp.Int('max_depth', 3, 10))
  else:
    model = linear_model.RidgeClassifier(
        alpha=hp.Float('alpha', 1e-3, 1, sampling='log'))
  return model

tuner = kt.tuners.SklearnTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('score', 'max'),
        max_trials=10),
    hypermodel=build_model,
    scoring=metrics.make_scorer(metrics.accuracy_score),
    cv=model_selection.StratifiedKFold(5),
    directory='.',
    project_name='my_project')

X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.2)

tuner.search(X_train, y_train)

best_model = tuner.get_best_models(num_models=1)[0]


INFO:tensorflow:Reloading Oracle from existing project .\my_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\my_project\tuner0.json
INFO:tensorflow:Oracle triggered exit


In [1]:
# global optimization to find coefficients for weighted ensemble on blobs problem
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from numpy import mean
from numpy import std
from numpy import array
from numpy import argmax
from numpy import tensordot
from numpy.linalg import norm
from scipy.optimize import differential_evolution

# fit model on dataset
def fit_model(trainX, trainy):
	trainy_enc = to_categorical(trainy)
	# define model
	model = Sequential()
	model.add(Dense(25, input_dim=2, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit model
	model.fit(trainX, trainy_enc, epochs=500, verbose=0)
	return model

# make an ensemble prediction for multi-class classification
def ensemble_predictions(members, weights, testX):
	# make predictions
	yhats = [model.predict(testX) for model in members]
	yhats = array(yhats)
	# weighted sum across ensemble members
	summed = tensordot(yhats, weights, axes=((0),(0)))
	# argmax across classes
	result = argmax(summed, axis=1)
	return result

# # evaluate a specific number of members in an ensemble
def evaluate_ensemble(members, weights, testX, testy):
	# make prediction
	yhat = ensemble_predictions(members, weights, testX)
	# calculate accuracy
	return accuracy_score(testy, yhat)

# normalize a vector to have unit norm
def normalize(weights):
	# calculate l1 vector norm
	result = norm(weights, 1)
	# check for a vector of all zeros
	if result == 0.0:
		return weights
	# return normalized vector (unit norm)
	return weights / result

# loss function for optimization process, designed to be minimized
def loss_function(weights, members, testX, testy):
	# normalize weights
	normalized = normalize(weights)
	# calculate error rate
	return 1.0 - evaluate_ensemble(members, normalized, testX, testy)

# generate 2d classification dataset
X, y = make_blobs(n_samples=1100, centers=3, n_features=2, cluster_std=2, random_state=2)
# split into train and test
n_train = 100
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]
print(trainX.shape, testX.shape)
# fit all models
n_members = 5
members = [fit_model(trainX, trainy) for _ in range(n_members)]
# evaluate each single model on the test set
testy_enc = to_categorical(testy)
for i in range(n_members):
	_, test_acc = members[i].evaluate(testX, testy_enc, verbose=0)
	print('Model %d: %.3f' % (i+1, test_acc))
# evaluate averaging ensemble (equal weights)
weights = [1.0/n_members for _ in range(n_members)]
score = evaluate_ensemble(members, weights, testX, testy)
print('Equal Weights Score: %.3f' % score)
# define bounds on each weight
bound_w = [(0.0, 1.0)  for _ in range(n_members)]
# arguments to the loss function
search_arg = (members, testX, testy)
# global optimization of ensemble weights
result = differential_evolution(loss_function, bound_w, search_arg, maxiter=1000, tol=1e-7)
# get the chosen weights
weights = normalize(result['x'])
print('Optimized Weights: %s' % weights)
# evaluate chosen weights
score = evaluate_ensemble(members, weights, testX, testy)
print('Optimized Weights Score: %.3f' % score)

ImportError: cannot import name 'to_categorical' from 'keras.utils' (C:\Users\furyx\miniconda3\envs\tf27\lib\site-packages\keras\utils\__init__.py)